<a href="https://colab.research.google.com/github/lordfiftyfive/OtherProjects/blob/master/AUTOMATEDTESTINGFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install robotframework

In [ ]:
*** Settings ***
Library               RequestsLibrary

*** Test Cases ***

Quick Get Request Test
    ${response}=    GET  https://www.google.com

Quick Get Request With Parameters Test
    ${response}=    GET  https://www.google.com/search  params=query=ciao  expected_status=200

Quick Get A JSON Body Test
    ${response}=    GET  https://jsonplaceholder.typicode.com/posts/1
    Should Be Equal As Strings    1  ${response.json()}[id]

In [ ]:
!pip install numba
#!pip install pytest
!pip install trl
!pip install transformers==4.24.0
!pip install pyrelational 
!pip install inferactively-pymdp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 880 kB 35.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8fcdad1bb3e75b86381db9e02a13733b2b081d187a1c06bd49de81294dece675
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.8 MB/s 
     |████████████████████████████████| 182 kB 43.6 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 
  Attempting uninstall: tokenizers
    Found existing instal

In [ ]:
import requests #for making rest api requests
import json
#RL
import torch
from transformers import GPT2Tokenizer
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
#active inference with relative distance strategy being the way we will implement diversity sampling 
from pyrelational.strategies.task_agnostic.relative_distance_strategy import (
    RelativeDistanceStrategy,
)

from pyrelational.models import LightningModel
from pyrelational.data import GenericDataManager

from scipy.spatial import distance

In [ ]:
# get models
gpt2_model = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

query_txt = "this morning I went to "#"00000000-0000-0000-0000-000000000000 "

query_tensor = gpt2_tokenizer.encode(query_txt, return_tensors="pt")
# get model response

response_tensor  = respond_to_batch(gpt2_model, query_tensor)
response_txt = gpt2_tokenizer.decode(response_tensor[0,:])
ep = 5
#reward is going to be -cosine similarity 
reward = -distance.cosine(query_tensor,response_tensor)# *0.5*correctness#-distance.euclidean(query_tensor,response_tensor)
#reward = -4*reward**2 + 8*reward
reward = [torch.tensor(reward)]
print(reward)

# initialize trainer
ppo_config = {'batch_size': 1, 'forward_batch_size': 1,'ppo_epochs':20}
ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer, **ppo_config)
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
response_tensor  = respond_to_batch(gpt2_model, query_tensor)
response_tx = gpt2_tokenizer.decode(response_tensor[0,:])
print(train_stats)
print("asdf")

print(response_tx)
print("fas")
print(response_txt)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['lm_head.weight', 'h.7.attn.masked_bias', 'v_head.summary.weight', 'h.1.attn.masked_bias', 'h.3.attn.masked_bias', 'h.8.attn.masked_bias', 'h.5.attn.masked_bias', 'h.2.attn.masked_bias', 'h.11.attn.masked_bias', 'h.10.attn.masked_bias', 'h.4.attn.masked_bias', 'h.6.attn.masked_bias', 'h.9.attn.masked_bias', 'h.0.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['lm_head.weight', 'h.7.attn.masked_bias', 'v_head.summary.weight', 'h.1.attn.masked_bias', 'h.3.attn.masked_bias', 'h.8.attn.masked_bias', 'h.5.attn.masked_bias', 'h.2.attn.masked_bias', 'h.11.attn.masked_bias', 'h.10.attn.masked_bias', 'h.4.attn.masked_bias', 'h.6.attn.masked_bias', 'h.9

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

ValueError: ignored

In [ ]:
dataset = 0

train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [400, 22, 20])
train_indices = train_ds.indices
val_indices = val_ds.indices
test_indices = test_ds.indices


data_manager = GenericDataManager(
    dataset=dataset,
    train_indices=train_indices,
    validation_indices=val_indices,
    test_indices=test_indices,
    hit_ratio_at=5,
)



model = LightningModel(gpt2_model,model_config={}, trainer_config={"epochs": 4})# this line compiles
strategy = RelativeDistanceStrategy(data_manager=data_manager, model=model)


NameError: ignored

In [ ]:
#this component is going to be for the part of the algorithm that chooses a numbers for the number entries
n = 10

b = max(b)

c = min(b)

for i in range(10):
  if i == 1:
    p = b
  elif i==2:
    p=c
  else: 
    p = b%2**i
  

In [ ]:

import requests
from requests_oauthlib import OAuth1
import json

#if dtype(u) == np.int: 
parameters = {
    "a": 's',
    "b": 'g'
}#GET /api/v4/{accessKey}/carts/customer
a = json.dumps(parameters)
print(a)
auth = OAuth1('de4cf57cfc45184510c627de9d01324ad13d95ddafc751024597bffa74af43ae4f793bdc6a8c5eb1c6364eb2')
rr = requests.head('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Carts/GetCartCustomer')
print(rr.headers)
rrr = requests.options('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Carts/GetCartCustomer')
print(rrr.content)
r = requests.post('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Carts/GetCartCustomer',auth=auth,data=parameters)#auth=('user', 'pass'))
#r.status_code()
#print(r.status_code)
print(r.json())
print(r.text())
rr = requests.head('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Carts/GetCartCustomer')
#r.text

#rr = requests.request('GET','https://loyaltyengine.bloyal.io/swagger/ui/index#/Cart/customer!'
#',auth=auth,data=json.dumps(payload))
#response = requests.get("https://api.open-notify.org/iss-pass.json", params=parameters)
#print(r.text)
#jprint(response.json())
#r = requests.get('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Cart/SetCartCustomer', params=payload)

#elif dtype(u) == string

{"a": "s", "b": "g"}
{'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html', 'Content-Encoding': 'gzip', 'Expires': '-1', 'Vary': 'Accept-Encoding', 'Server': 'Microsoft-IIS/10.0', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Fri, 02 Dec 2022 22:52:03 GMT'}
b'<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="UTF-8">\r\n  <title>Swagger UI</title>\r\n  <link rel="icon" type="image/png" href="images/favicon-32x32-png" sizes="32x32" />\r\n  <link rel="icon" type="image/png" href="images/favicon-16x16-png" sizes="16x16" />\r\n  <link href=\'css/typography-css\' media=\'screen\' rel=\'stylesheet\' type=\'text/css\'/>\r\n  <link href=\'css/reset-css\' media=\'screen\' rel=\'stylesheet\' type=\'text/css\'/>\r\n  <link href=\'css/screen-css\' media=\'screen\' rel=\'stylesheet\' type=\'text/css\'/>\r\n  <link href=\'css/reset-css\' media=\'print\' rel=\'stylesheet\' type=\'text/css\'/>\r\n  <link href=\'css/p

JSONDecodeError: ignored

In [ ]:
#this is all going to be part of a for loop that will for the trl code

import requests
from requests_oauthlib import OAuth1
import json
w = 50
for i in range(w):
  #if dtype(u) == np.int: 
  parameters = {
      "a": 's',
      "b": 'g'
  }#GET /api/v4/{accessKey}/carts/customer
  a = json.dumps(parameters)
  print(a)
  auth = OAuth1('de4cf57cfc45184510c627de9d01324ad13d95ddafc751024597bffa74af43ae4f793bdc6a8c5eb1c6364eb2')
  r = requests.post('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Carts/GetCartCustomer',auth=auth,data=parameters)#auth=('user', 'pass'))
  r.status_code
  print(r.status_code)
  #r.text

  #rr = requests.request('GET','https://loyaltyengine.bloyal.io/swagger/ui/index#/Cart/customer!'
  #',auth=auth,data=json.dumps(payload))
  #response = requests.get("https://api.open-notify.org/iss-pass.json", params=parameters)
  #print(r.text)
  #jprint(response.json())
  #r = requests.get('https://loyaltyengine.bloyal.io/swagger/ui/index#!/Cart/SetCartCustomer', params=payload)
  r.json()
  #elif dtype(u) == string
#r.json()
#print(response)

{"a": "s", "b": "g"}
200


JSONDecodeError: ignored